In [4]:
import pandas as pd
import pandahouse as ph

from datetime import datetime

#import seaborn as sns
#import matplotlib.pyplot as plt
#import numpy as np
#import re
#%matplotlib inline

In [5]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
'database':'default',
'user':'student',
'password':'dpo_python_2020'
}

In [6]:
logistic_online = pd.read_csv('logistic_online.csv')

q2 = '''
SELECT
    *
FROM
    default.product_online
'''
product_online = ph.read_clickhouse(query=q2, connection=connection_default)

q3 = '''
SELECT
    *
FROM
    default.returns_online
'''
returns_online = ph.read_clickhouse(query=q3, connection=connection_default)


# Предобработка данных

# logistic_online

In [7]:
df_log = logistic_online

**date_order** - дата, в которую клиент оформляет заказ

**number_order** - уникальный номер заказа клиента (в столбце номер может повторяться, т.к. заказы переносятся и имеют раные даты доставки)

**timeslot_number** - нумерация кол-ва попыток доставить в плановую дату доставки. Если доставка не была осуществлена, то заказ переносится на новую дату

**result_data_order** - дата, в которую заказ был доставлен клиенту

**delivery_type** - выбранный клиентом способ получения заказа (курьерка или самовывоз)

**carrier_name** - перевозчик, которым заказ доставляется клиенту

**seller_ID** - уникальный идентификатор продавца

**order_status** - последний актуальный статус заказа (фиксируется во всех строках последний статус заказа)

**customer_cluster** - региональная принадлежность клиента

**warehouse_cluster** - региональная принадлежность продавца

                                                Смотрим датафрейм

In [8]:
df_log.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-5-01,313655114,2021-14-01,1,2021-11-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-3-01,315229153,2021-11-01,1,2021-8-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-8-01,3114106095,2021-16-01,1,2021-14-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-12-01,31145541407,2021-20-01,1,2021-15-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-16-01,325050136,2021-26-01,1,2021-25-01,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [9]:
df_log.dtypes

date_order            object
number_order           int64
planned_data_order    object
timeslot_number        int64
result_data_order     object
delivery_type         object
carrier_name          object
seller_ID              int64
order_status          object
customer_cluster      object
warehouse_cluster     object
dtype: object

In [10]:
df_log.isna().sum()

date_order            625
number_order            0
planned_data_order      0
timeslot_number         0
result_data_order     890
delivery_type           0
carrier_name            0
seller_ID               0
order_status            0
customer_cluster        0
warehouse_cluster       0
dtype: int64

In [11]:
df_log.customer_cluster.value_counts()

Кластер Москва                1808
Кластер Екатеринбург          1133
Кластер Московская область     843
Кластер Краснодар              710
Кластер Санкт-Петербург        565
Кластер Казань                 512
Кластер Центр                  472
Кластер Новосибирск            225
Кластер Северо-Запад           200
Кластер Хабаровск               58
Кластер Калининград             37
Кластер Якутия                  34
Name: customer_cluster, dtype: int64

In [12]:
df_log.warehouse_cluster.value_counts()

Кластер Москва                1752
Кластер Екатеринбург          1148
Кластер Московская область    1075
Кластер Казань                 617
Кластер Краснодар              588
Кластер Санкт-Петербург        470
Кластер Новосибирск            390
Кластер Центр                  246
Кластер Северо-Запад           205
Кластер Калининград             96
Кластер Хабаровск               10
Name: warehouse_cluster, dtype: int64

In [13]:
df_log.delivery_type.value_counts()

Курьер       3502
Самовывоз    3095
Name: delivery_type, dtype: int64

In [14]:
df_log.carrier_name.value_counts()

Самодоставка    2109
3PL 2           1890
3PL 1           1827
3PL 3            517
3PL 4            254
Name: carrier_name, dtype: int64

In [15]:
df_log.order_status.value_counts()

Доставлен            5694
На пути к клиенту     423
Отменен               334
Ожидает сборки        142
Спор                    4
Name: order_status, dtype: int64

                                                    ВЫВОД
Необходимо привести даты к типу datetime в 3 колонках.

In [16]:
df_log = df_log.reset_index()

In [17]:
def correct_date(df, column):
    date_list1 = list()
    for i in df[column]:
        date = i.split('-')
            # исправляем день месяца (добавляем 0 в начало)
        if int(date[1]) < 10 and len(date[1]) > 2:
            date[1] = '0' + date[1]
            # исправляем номер месяца (оставляем 2 цифры)
        if len(date[2]) == 3:
            date[2] = date[2][1:]
        date1 = pd.to_datetime(date[0] + '-' + date[2] + '-' + date[1])
        date_list1.append(date1)
    return date_list1


In [18]:
#Датафрейм для корректировки date_order
df_date_order = df_log[['index','date_order']].dropna()

#Датафрейм для корректировки дат date_order 
df_date_order['correct_date_order'] = correct_date(df_date_order, 'date_order')
df_date_order = df_date_order.drop(columns=['date_order'])


In [19]:
#Датафрейм для корректировки date_order
df_planned_data_order = df_log[['index','planned_data_order']].dropna()

#Датафрейм для корректировки дат date_order 
df_planned_data_order['correct_planned_data_order'] = correct_date(df_planned_data_order, 'planned_data_order')
df_planned_data_order = df_planned_data_order.drop(columns=['planned_data_order'])

In [20]:
#Датафрейм для корректировки date_order
df_result_data_order = df_log[['index','result_data_order']].dropna()

#Датафрейм для корректировки дат date_order 
df_result_data_order['correct_result_data_order'] = correct_date(df_result_data_order, 'result_data_order')
df_result_data_order = df_result_data_order.drop(columns=['result_data_order'])


In [21]:
# Добавляем колонки с исправленными датами к исходному датафрейму
df_log = df_log.merge(df_date_order, on='index', how='left')
df_log = df_log.merge(df_planned_data_order, on='index', how='left')
df_log = df_log.merge(df_result_data_order, on='index', how='left')

In [22]:
df_log.head()

,index,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,correct_date_order,correct_planned_data_order,correct_result_data_order
0,0,2021-5-01,313655114,2021-14-01,1,2021-11-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва,2021-01-05,2021-01-14,2021-01-11
1,1,2021-3-01,315229153,2021-11-01,1,2021-8-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,2021-01-03,2021-01-11,2021-01-08
2,2,2021-8-01,3114106095,2021-16-01,1,2021-14-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,2021-01-08,2021-01-16,2021-01-14
3,3,2021-12-01,31145541407,2021-20-01,1,2021-15-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва,2021-01-12,2021-01-20,2021-01-15
4,4,2021-16-01,325050136,2021-26-01,1,2021-25-01,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург,2021-01-16,2021-01-26,2021-01-25


In [23]:
# Приводим датафрейм к исходному виду с исправленными датами
df_log = df_log.drop(columns=['index', 'date_order', 'planned_data_order', 'result_data_order'])

df_log = df_log[['correct_date_order', 'number_order', 'correct_planned_data_order',
                'timeslot_number', 'correct_result_data_order', 'delivery_type', 'carrier_name',
                'seller_ID', 'order_status', 'customer_cluster', 'warehouse_cluster']]

df_log = df_log.rename(columns={'correct_date_order'        : 'date_order', 
                                'correct_planned_data_order': 'planned_data_order', 
                                'correct_result_data_order' : 'result_data_order'})

In [24]:
df_log.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


# product_online

In [25]:
df_prod = product_online

**number_order** - уникальный номер заказа клиента (в столбце номер может повторяться, т.к. так как в одном заказе может быть несколько товаров)

**product_ID** - уникальный номер товара (В столбце номер может повторяться, т.к. так как один товар может быть в заказах нескольких клиентов)

**product_price** - стоимость одной единицы товара в рублях

**amount** - количество единиц товара в заказе

In [26]:
df_prod

,number_order,product_ID,product_price,amount
0,313655114,983587801,5512.44,1
1,313655114,983587158,4070.35,2
2,313655114,983582669,3591.97,1
3,315229153,983589424,4433.23,3
4,315229153,983587827,2862.50,2
...,...,...,...,...
12228,3649144391,983583847,8146.30,2
12229,3649144391,983581209,2816.34,4
12230,365435447,983582430,2937.91,4
12231,365435447,983589903,7626.13,4


In [27]:
df_prod.dtypes

number_order       int64
product_ID         int64
product_price    float64
amount             int16
dtype: object

In [28]:
df_prod.isna().sum()

number_order     0
product_ID       0
product_price    0
amount           0
dtype: int64

# returns_online

In [29]:
df_prod = product_online

In [30]:
df_prod 

,number_order,product_ID,product_price,amount
0,313655114,983587801,5512.44,1
1,313655114,983587158,4070.35,2
2,313655114,983582669,3591.97,1
3,315229153,983589424,4433.23,3
4,315229153,983587827,2862.50,2
...,...,...,...,...
12228,3649144391,983583847,8146.30,2
12229,3649144391,983581209,2816.34,4
12230,365435447,983582430,2937.91,4
12231,365435447,983589903,7626.13,4


In [31]:
df_prod.dtypes

number_order       int64
product_ID         int64
product_price    float64
amount             int16
dtype: object

In [32]:
df_prod.isna().sum()

number_order     0
product_ID       0
product_price    0
amount           0
dtype: int64

# ЗАДАЧА 1.

Посчитать **OnTime** - Метрика своевременности доставки. (Доставили в плановую дату доставки или нет), в %

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада, по таймслотам;